In [85]:
import pandas as pd
dataset=pd.read_csv("final_dataset_basicmlmodel.csv")

In [86]:
dataset.head(2)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...


In [87]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,".",tweet)

1 .  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 . we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 . i get to see my daddy today!!   #80days #gettingfed
4 . ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 . i am thankful for having a paner. #thankful #positive     


In [88]:
import re

In [89]:


#Clean text from noise
def clean_text(text):
    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    
    #Convert to lowercase to maintain consistency
    text = text.lower()
       
    return text
    

In [90]:
dataset["clean_text"]=dataset.tweet.apply(lambda x: clean_text(x))

In [91]:
STOP_WORDS = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'also', 'am', 'an', 'and',
              'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
              'between', 'both', 'but', 'by', 'can', "can't", 'cannot', 'com', 'could', "couldn't", 'did',
              "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'else', 'ever',
              'few', 'for', 'from', 'further', 'get', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having',
              'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 'his', 'how',
              "how's", 'however', 'http', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it',
              "it's", 'its', 'itself', 'just', 'k', "let's", 'like', 'me', 'more', 'most', "mustn't", 'my', 'myself',
              'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'otherwise', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'r', 'same', 'shall', "shan't", 'she', "she'd", "she'll", "she's",
              'should', "shouldn't", 'since', 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs',
              'them', 'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", "they're",
              "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't",
              'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
              "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't",
              'www', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

In [92]:
def gen_freq(text):
    #Will store the list of words
    word_list = []

    #Loop over all the tweets and extract words into word_list
    for tw_words in text.split():
        word_list.extend(tw_words)

    #Create word frequencies using word_list
    word_freq = pd.Series(word_list).value_counts()
    
    #Drop the stopwords during the frequency calculation
    word_freq = word_freq.drop(STOP_WORDS, errors='ignore')
    
    return word_freq

#Check whether a negation term is present in the text
def any_neg(words):
    for word in words:
        if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
            return 1
        else:
            return 0

#Check whether one of the 100 rare words is present in the text
def any_rare(words, rare_100):
    for word in words:
        if word in rare_100:
            return 1
        else:
            return 0

#Check whether prompt words are present
def is_question(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
        else:
            return 0

In [93]:
word_freq=gen_freq(dataset.clean_text.str)

In [94]:
rare_100=word_freq[-100]

In [95]:
dataset.head(2)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...


In [96]:
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x: len(x))
#dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x: any_neg(x))
#dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x: is_question(x))
#dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x: is_question(x))
#dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
#dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
#dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))
dataset['char_count'] = dataset.clean_text.apply(lambda x: len(x))
#word_freq = gen_fre(dataset.clean_text.str)
#100 most rare words in the dataset
#rare_100 = word_freq[-100:]
#Number of words in a tweet

#Negation present or not

#Prompt present or not

#Any of the most 100 rare words present or not

#Character count of the tweet


In [97]:
#gen_freq(dataset.clean_text.str)[:10]
gen_freq(dataset.clean_text.str)[:10]

user      3351
amp        439
love       320
day        254
trump      214
happy      207
will       191
people     186
new        171
u          158
dtype: int64

In [98]:
gen_freq(dataset.tweet.str)[:10]

@user    3284
&amp;     388
â¦       229
-         195
will      187
happy     171
#love     155
day       154
new       151
u         144
dtype: int64

In [99]:
dataset.head(3)

,id,label,tweet,clean_text,word_count,any_neg,is_question,char_count
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...,18,0,0,102
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...,19,0,0,122
2,3,0,bihday your majesty,bihday your majesty,3,0,0,21


In [100]:
#from sklearn.model_selection import train_test_split

#X = dataset[['word_count', 'any_neg', 'any_rare', 'char_count', 'is_question']]
#y = dataset.label

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=27)
from sklearn.model_selection import train_test_split
X=dataset[['word_count','any_neg','char_count','is_question']]
y=dataset.label
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=27)

In [101]:
#from sklearn.naive_bayes import GaussianNB

#Initialize GaussianNB classifier
#model = GaussianNB()
#Fit the model on the train dataset
#model = model.fit(X_train, y_train)
#Make predictions on the test dataset
#pred = model.predict(X_test)
from sklearn.naive_bayes  import GaussianNB
model=GaussianNB()
model=model.fit(X_train,y_train)
pred=model.predict(X_test)

In [102]:
#from sklearn.metrics import accuracy_score

#print("Accuracy:", accuracy_score(y_test, pred)*100, "%")
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,pred)*100,"%")

Accuracy: 60.76190476190476 %
